In [3]:
###fnc definitions, import data
import jax
import jax.numpy as jnp
from jax import jit
import jax.random as rnd
# from jax.experimental.host_callback import id_print
# from jax.experimental.host_callback import call
import optax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
# matplotlib.use('Agg')
from drawnow import drawnow
import numpy as np
import csv
import pickle
from pathlib import Path
from datetime import datetime
import re
import os
import sys
# from os.path import dirname, abspath
import scipy

def load_(str_):
    path_ = os.path.dirname(os.getcwd()) + '/pkl/' # Path(__file__).resolve().parents[1]
    with open(path_+str_,'rb') as file_:
        # param = pickle.load(file_)
        param_ = jnp.load(file_,allow_pickle=True)
    return param_

def geod_dist(dots,pos):### calculate geodesic
    pos_ = pos #(pos+jnp.pi)%(2*jnp.pi)-jnp.pi # could cause problems...
    th1 = jnp.minimum(jnp.abs(pos_[1]-dots[:,1]),2*jnp.pi-jnp.abs(pos_[1]-dots[:,1]))
    th2 = jnp.minimum(jnp.abs(pos_[0]-dots[:,0]),2*jnp.pi-jnp.abs(pos_[0]-dots[:,0]))
    return jnp.sqrt(th1**2+th2**2)

vals_train,vals_test,theta_test = load_('v9_all_25_04-1610.pkl')
(R_tot,R_obj,R_env,R_dot,R_sel),(sd_tot,sd_obj,sd_env,sd_dot,sd_sel) = vals_train
(pos_arr,switch_arr,R_all) = vals_test
(R_test,R_r,R_g,R_b) = R_all # [TESTS,VMAPS,IT]

ki = rnd.split(rnd.PRNGKey(0),num=30)

print(R_tot.shape)

(2000,)


In [7]:
#find reward intervals
TESTS = R_test.shape[0]
VMAPS = R_test.shape[1]
IT = R_test.shape[2]
threshold = 0.6

# get dots

# get selected dot
# find geod_dist to dot

intervals = np.empty((TESTS,VMAPS,IT//2))
sel_dot = np.empty((TESTS,VMAPS,2))
dis = np.empty((TESTS,VMAPS))
for t in range(TESTS):
    for v in range(VMAPS):
        dots = theta_test["ENV"]["DOTS"][t,:,:,v]
        sel = theta_test["ENV"]["SELECT"][t,v,:]
        sel_dot[t,v,:] = np.dot(sel,dots)
        dis[t,v] = np.norm(sel_dot[t,v,:])
        obj_peaks = scipy.signal.find_peaks((-R_test[t,v,:]),height=threshold)
        if obj_peaks[0].size == 0:
            continue
        else:
            intervals[t,v,:len(obj_peaks[0])-1] = np.diff(obj_peaks[0])

        r_peaks_ = scipy.signal.find_peaks((-R_r[t,v,:]),height=threshold)
        g_peaks = scipy.signal.find_peaks((-R_g[t,v,:]),height=threshold)
        b_peaks = scipy.signal.find_peaks((-R_b[t,v,:]),height=threshold)

avg_intervals = np.mean(intervals,axis=(0,1))
sd_intervals = np.std(intervals,axis=(0,1))
int_hist = np.histogram(avg_intervals)

#plot
fig,ax = plt.subplots()
ax.errorbar(np.arange(len(avg_intervals)),avg_intervals,yerr=sd_intervals/2)
# ax.hist(avg_intervals,bins=20)
ax.set_xlabel('Reward Interval')
ax.set_ylabel('Frequency')
ax.set_title('Reward Interval Distribution')
plt.show()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [ ]:
#(replot training)
plt.figure()
# title__ = f'v9 training, tot epochs={TOT_EPOCHS}, it={IT}, vmaps={VMAPS}, init={INIT:.2f}, update={UPDATE:.5f}, SIGMA_A={SIGMA_A:.1f}, SIGMA_R0={SIGMA_R0:.1f}, SIGMA_RINF={SIGMA_RINF:.1f}, \n SIGMA_N={SIGMA_N:.1f}, STEP={STEP:.3f} WD={WD:.5f}, LAMBDA_D={LAMBDA_D:.4f}, LAMBDA_E={LAMBDA_E:.4f}, LAMBDA_S={LAMBDA_S:.4f}, NEURONS={NEURONS}' # \n colors={jnp.array_str(COLORS[0][:]) + jnp.array_str(COLORS[1][:]) + jnp.array_str(COLORS[2][:])}' #  + jnp.array_str(COLORS[3][:]) + jnp.array_str(COLORS[4][:])}'
fig,ax = plt.subplots(2,3,figsize=(16,9))
# plt.suptitle(title__,fontsize=14)
plt.subplot(2,3,1)
plt.errorbar(jnp.arange(len(R_tot)),R_tot,yerr=sd_tot/2,ecolor="black",elinewidth=0.5,capsize=1.5)
plt.ylabel(r'$R_{tot}$',fontsize=15)
plt.xlabel(r'Iteration',fontsize=12)
plt.subplot(2,3,2)
plt.errorbar(jnp.arange(len(R_obj)),R_obj,yerr=sd_obj/2,ecolor="black",elinewidth=0.5,capsize=1.5)
plt.ylabel(r'$R_{obj}$',fontsize=15)
plt.xlabel(r'Iteration',fontsize=12)
plt.subplot(2,3,3)
plt.errorbar(jnp.arange(len(R_env)),R_env,yerr=sd_env/2,ecolor="black",elinewidth=0.5,capsize=1.5)
plt.ylabel(r'$R_{env}$',fontsize=15)
plt.xlabel(r'Iteration',fontsize=12)
plt.subplot(2,3,4)
plt.errorbar(jnp.arange(len(R_dot)),R_dot,yerr=sd_dot/2,ecolor="black",elinewidth=0.5,capsize=1.5)
plt.ylabel(r'$R_{dot}$',fontsize=15)
plt.xlabel(r'Iteration',fontsize=12)
plt.subplot(2,3,5)
plt.errorbar(jnp.arange(len(R_sel)),R_sel,yerr=sd_sel/2,ecolor="black",elinewidth=0.5,capsize=1.5)
plt.ylabel(r'$R_{sel}$',fontsize=15)
plt.xlabel(r'Iteration',fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
#(replot testing)
geod_dist_vmap = jax.vmap(geod_dist,in_axes=(None,0),out_axes=0)# (e_t,pos_t)
colors_ = np.float32([[255,0,0],[0,255,0],[0,0,255]])/255 #theta_0["ENV"]["COLORS"]/255
colormap = cm.seismic(np.linspace(0,1,IT+1), alpha=1)

plt.figure()
# title__ = f'v9 testing, tot epochs={TOT_EPOCHS}, it={IT}, vmaps={VMAPS}, init={INIT:.2f}, update={UPDATE:.5f}, SIGMA_N={SIGMA_N}, SIGMA_A={SIGMA_A:.1f}, SIGMA_R0={SIGMA_R0:.1f}, SIGMA_RINF={SIGMA_RINF:.1f}, \n SIGMA_N={SIGMA_N:.1f}, STEP={STEP:.3f} WD={WD:.5f}, LAMBDA_D={LAMBDA_D:.4f}, LAMBDA_E={LAMBDA_E:.4f}, LAMBDA_S={LAMBDA_S:.4f}, NEURONS={NEURONS}'
fig,axis = plt.subplots(2*TESTS,4,figsize=(15,5*TESTS+2))#(4,5)
# plt.suptitle(title__,fontsize=14)
for i in range(TESTS):
    k = rnd.randint(ki[18+i],(),0,VMAPS) # rnd.choice(ki[18+j],loop_params["VMAPS"],replace=False)
    ax0 = plt.subplot2grid((2*TESTS,4),(2*i,2),colspan=2)
    ax0.set_ylabel(r'$dis$',fontsize=16)
    # dis_old = dis_arr[i,k,:,:] # [TESTS,VMAPS,IT,3]
    ax0.tick_params(axis='both', which='major', labelsize=14)
    ax0.set_ylim(-0.2,jnp.pi+0.2) # 2*jnp.pi*jnp.sqrt(2)

    ax1 = plt.subplot2grid((2*TESTS,4),(2*i,0),rowspan=2,colspan=2)
    pos_ = pos_arr[i,k,:,:] # [TESTS,VMAPS,IT,2]
    ax1.scatter(pos_[:,0],pos_[:,1],s=30,color=colormap,marker='.')# axis[i,1]

    sel_ = theta_test["ENV"]["SELECT"][i,k,:] # [EPOCHS,VMAPS,3]
    dots_ = theta_test["ENV"]["DOTS"][i,:,:,k] # [EPOCHS,3,2,VMAPS]
    dis_ = geod_dist_vmap(dots_,pos_)
    ax2 = plt.subplot2grid((2*TESTS,4),(2*i+1,2),colspan=2)
    ax2.plot(R_test[i,k,1:],color='k',linewidth=2) #axis[i,2]
    ax2.set_ylabel(r'$R_{tot}$',fontsize=16)
    ax2.tick_params(axis='both', which='major', labelsize=14)
    ax2.set_ylim(-1.1,0.1) # -1.1,0 -jnp.sqrt(2*jnp.pi)*
    for j in range(theta_test["ENV"]["N_DOTS"]):
        ax0.plot(dis_[1:,j],color=(colors_[j,:]),linewidth=3)#axis[i,0], tuple
        ax0.plot(dis_[1:,j],color=(colors_[j,:]),alpha=0.5,linewidth=1)
        ax1.scatter(dots_[j,0],dots_[j,1],s=60,marker='x',color=(colors_[j,:]))

    ax1.set_xlim(-jnp.pi,jnp.pi)
    ax1.set_ylim(-jnp.pi,jnp.pi)
    ax1.set_xticks([-jnp.pi,-jnp.pi/2,0,jnp.pi/2,jnp.pi])
    ax1.set_xticklabels(['$-\pi$','$-\pi/2$','0','$\pi/2$','$\pi$'],fontsize=14)
    ax1.set_yticks([-jnp.pi,-jnp.pi/2,0,jnp.pi/2,jnp.pi])
    ax1.set_yticklabels(['$-\pi$','$-\pi/2$','0','$\pi/2$','$\pi$'],fontsize=14)
    ax1.set_aspect('equal')
    ax1.set_title(f'sel={sel_}',fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.94) 